In [1]:
import numpy as np
import pandas as pd

from align import *

In [2]:
! python -V

Python 3.7.7


In [3]:
np.__version__, pd.__version__

('1.21.5', '1.3.5')

# Example of variant ID matching

This script is to handle to major-minor allele swap between hg19 and hg38.

Variants are on the "+" chian; Variants are left-normalized. (Using bcftools)

* p1: ref == ref, alt == alt
* p2: ref == alt, alt == ref (major minor allele swap)
* p3: not matched

For SNV, p1 and p2 should have no overlapping; for indel, p1 and p2 may overlap.

The reason is, if the sequence is CAC, and the record is CAC/C and C/CAC, it is unclear they are the same or not:

(1) Ref: CAC, Alt: C; deletion

(2) Ref: CAC, Alt: CACAC; the normalized record will be C/CAC, insertion.

One way is to use only p1; another way is to remove such ambiguous sites. However, by ploting the MAF, it should be able to know whether there are some mistakes or not.

In [4]:
df1 = pd.read_table("../../data/sample1.snv.txt")
df2 = pd.read_table("../../data/sample2.snv.txt")

In [5]:
df1

,w_pos,w_ref,w_alt
0,93335,G,T
1,106167,C,G
2,219975,C,T
3,225452,G,A
4,226672,G,A
...,...,...,...
581179,58606597,G,A
581180,58606615,T,C
581181,58606661,T,A
581182,58606673,A,C


In [6]:
df2

,t_pos,t_ref,t_alt
0,93335.0,G,T
1,106167.0,C,G
2,219975.0,C,T
3,225452.0,G,A
4,226672.0,G,A
...,...,...,...
258335,55132957.0,C,A
258336,55188086.0,C,T
258337,55729899.0,G,A
258338,57492212.0,G,A


In [7]:
df = merge_multi_allelic(df1,df2,"w_pos","t_pos","w_ref","t_ref","w_alt","t_alt","sample.snv",vtype="snv")

581184 variants in df1, 258340 variants in df2
257009 unique variant with ref==ref, alt==alt (pattern 1)
257009 records matched. Generally, number of records should equal the number of variants
1330 unique variant with ref==alt, alt==ref (pattern 2)
1330 records matched. Generally, number of records should equal the number of variants
however, liftover may lift different variants to the same position. Please take caution if variants != records.
322845 variants not matched
sample.snv_p1p2.txt.gz contains overlapping variants to calculate dosage r^2
sample.snv_p1p2p3.txt contains overlapping + df1 unique variants to calculate the coverage
=== final report ===
581184 variants in df1
257009 variants in df1 matched to df2 with ref==ref, alt==alt
1330 variants in df1 matched to df2 with ref==alt, alt==ref
crude coverage: 258339 / 581184


In [8]:
df

,w_pos,w_ref,w_alt,t_pos,t_ref,t_alt,pattern
index,,,,,,,
0,93335,G,T,93335.0,G,T,1
1,106167,C,G,106167.0,C,G,1
2,219975,C,T,219975.0,C,T,1
3,225452,G,A,225452.0,G,A,1
4,226672,G,A,226672.0,G,A,1
...,...,...,...,...,...,...,...
258334,54911735,T,C,54911735.0,C,T,2
258335,55132957,A,C,55132957.0,C,A,2
258336,55188086,T,C,55188086.0,C,T,2


In [9]:
df3 = pd.read_table("../../data/sample1.indel.txt")
df4 = pd.read_table("../../data/sample2.indel.txt")

In [10]:
df3

,w_pos,w_ref,w_alt
0,261963,TTGTC,T
1,264587,GAC,G
2,267222,ACGCTGCGCTG,A
3,274615,GTTTAT,G
4,282987,GTC,G
...,...,...,...
65152,58600716,ACGG,A
65153,58604967,ATCT,A
65154,58605004,A,AT
65155,58605396,A,ACCCTAGAACTCCT


In [11]:
df4

,t_pos,t_ref,t_alt
0,261963.0,TTGTC,T
1,264587.0,GAC,G
2,267222.0,ACGCTGCGCTG,A
3,274615.0,GTTTAT,G
4,282987.0,GTC,G
...,...,...,...
16830,56188781.0,AAAATAAATAAATAAATAAAT,A
16831,56659130.0,CAATAAT,C
16832,58018301.0,AGT,A
16833,58161424.0,TCAAACAAA,T


In [12]:
df = merge_multi_allelic(df3,df4,"w_pos","t_pos","w_ref","t_ref","w_alt","t_alt","sample.indel",vtype="indel")

65157 variants in df1, 16835 variants in df2
16444 unique variant with ref==ref, alt==alt (pattern 1)
16444 records matched. Generally, number of records should equal the number of variants
740 unique variant with ref==alt, alt==ref (pattern 2)
740 records matched. Generally, number of records should equal the number of variants
however, liftover may lift different variants to the same position. Please take caution if variants != records.
222 multi-allelic sites show both pattern 1 and pattern 2
48195 variants not matched
sample.indel_p1p2.txt.gz contains overlapping variants to calculate dosage r^2
sample.indel_p1p2p3.txt contains overlapping + df1 unique variants to calculate the coverage
=== final report ===
65157 variants in df1
16444 variants in df1 matched to df2 with ref==ref, alt==alt
740 variants in df1 matched to df2 with ref==alt, alt==ref
crude coverage: 16962 / 65157


In [24]:
df

,w_pos,w_ref,w_alt,t_pos,t_ref,t_alt,pattern
index,,,,,,,
0,261963,TTGTC,T,261963.0,TTGTC,T,1
1,264587,GAC,G,264587.0,GAC,G,1
2,267222,ACGCTGCGCTG,A,267222.0,ACGCTGCGCTG,A,1
3,274615,GTTTAT,G,274615.0,GTTTAT,G,1
4,282987,GTC,G,282987.0,GTC,G,1
...,...,...,...,...,...,...,...
17068,58030372,A,AAAC,58030372.0,AAAC,A,2
17069,58062353,C,CTTAT,58062353.0,CTTAT,C,2
17070,58163331,A,AAAAT,58163331.0,AAAAT,A,2


222 multi-allelic sites show both pattern 1 and pattern 2, have a look:

In [20]:
tmp1 = df3.merge(df4, how="inner", left_on=["w_pos", "w_ref", "w_alt"], right_on=["t_pos", "t_ref", "t_alt"])
tmp2 = df3.merge(df4, how="inner", left_on=["w_pos", "w_ref", "w_alt"], right_on=["t_pos", "t_alt", "t_ref"])

In [23]:
tmp1.merge(tmp2, how="inner", on=["w_pos", "w_ref", "w_alt"])

,w_pos,w_ref,w_alt,t_pos_x,t_ref_x,t_alt_x,t_pos_y,t_ref_y,t_alt_y
0,865663,GC,G,865663.0,GC,G,865663.0,G,GC
1,955709,AG,A,955709.0,AG,A,955709.0,A,AG
2,1046749,A,AG,1046749.0,A,AG,1046749.0,AG,A
3,1046749,AG,A,1046749.0,AG,A,1046749.0,A,AG
4,1395488,G,GCGC,1395488.0,G,GCGC,1395488.0,GCGC,G
...,...,...,...,...,...,...,...,...,...
106,56406702,AT,A,56406702.0,AT,A,56406702.0,A,AT
107,56406702,A,AT,56406702.0,A,AT,56406702.0,AT,A
108,56676320,AAAAC,A,56676320.0,AAAAC,A,56676320.0,A,AAAAC
109,57054751,G,GT,57054751.0,G,GT,57054751.0,GT,G


For example, at site 1046749, ref allele is AG, alt alleles are A or AGG. Comparing the allele frequency would help.

In my work, I used the 16444 variants with pattern 1 and checked the AF. It will slightly make the coverage lower, but make the comparision more confident.